# Buff Training Dataset

In [1]:
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd 
import os

## Import Data

In [2]:
df = pd.read_csv("../data/train_buff/0000-train_buff.csv")
print(df.head())

   id                                                url   topic_folder  \
0   0  https://www.wa.de/nordrhein-westfalen/cannabis...  cannabis_html   
1   1  https://www.ak-kurier.de/akkurier/www/artikel/...  cannabis_html   
2   2  https://www.aponet.de/artikel/weniger-alkoholp...  cannabis_html   
3   3  https://www.nordbayern.de/panorama/norwegische...  cannabis_html   
4   4  https://www.deutschlandfunk.de/legalisierung-c...  cannabis_html   

  date_folder                                           filename  \
0       Feb23  Legalisierung von Cannabis_ Hanf aus NRW könnt...   
1       Feb23  Wann wird Cannabis in Deutschland legal_ _ AK-...   
2       Feb23  Weniger Alkoholprobleme bei Legalisierung von ...   
3       Feb23  Norwegische Studie_ Legalisierung von Cannabis...   
4       Feb23  Kehrtwende in Drogenpolitik - Der lange Weg zu...   

                                           full_path  
0  cannabis_html/Feb23/Legalisierung von Cannabis...  
1  cannabis_html/Feb23/Wann wi

## Extract Content from HTML Files

In [3]:
path_prefix = "../data/train_buff/"
extracted_texts = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    file_path = path_prefix + row['full_path']
    
    # Ensure the file exists (optional, based on your use case)
    if os.path.exists(file_path):
        
        # Open and read the HTML file
        with open(file_path, 'r', encoding='utf-8') as file:
            html_content = file.read()
        
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(html_content, 'html.parser') 
        
        text_content = soup.get_text(separator=' ', strip=True)
        extracted_texts.append(text_content)
        #print(text_content)
        
        # Example: Find and print the title tag content
        #print(soup.title.text)
    else:
        print(f"File not found: {file_path}")
        extracted_texts.append(None)

df['extracted_text'] = extracted_texts

100%|██████████| 311/311 [00:19<00:00, 15.88it/s]


In [4]:
df['topic'] = df['topic_folder'].str.replace('_html', '')

In [5]:
df.head()

,id,url,topic_folder,date_folder,filename,full_path,extracted_text,topic
0,0,https://www.wa.de/nordrhein-westfalen/cannabis...,cannabis_html,Feb23,Legalisierung von Cannabis_ Hanf aus NRW könnt...,cannabis_html/Feb23/Legalisierung von Cannabis...,Legalisierung von Cannabis: Hanf aus NRW könnt...,cannabis
1,1,https://www.ak-kurier.de/akkurier/www/artikel/...,cannabis_html,Feb23,Wann wird Cannabis in Deutschland legal_ _ AK-...,cannabis_html/Feb23/Wann wird Cannabis in Deut...,Wann wird Cannabis in Deutschland legal? | AK-...,cannabis
2,2,https://www.aponet.de/artikel/weniger-alkoholp...,cannabis_html,Feb23,Weniger Alkoholprobleme bei Legalisierung von ...,cannabis_html/Feb23/Weniger Alkoholprobleme be...,Weniger Alkoholprobleme bei Legalisierung von ...,cannabis
3,3,https://www.nordbayern.de/panorama/norwegische...,cannabis_html,Feb23,Norwegische Studie_ Legalisierung von Cannabis...,cannabis_html/Feb23/Norwegische Studie_ Legali...,Norwegische Studie: Legalisierung von Cannabis...,cannabis
4,4,https://www.deutschlandfunk.de/legalisierung-c...,cannabis_html,Feb23,Kehrtwende in Drogenpolitik - Der lange Weg zu...,cannabis_html/Feb23/Kehrtwende in Drogenpoliti...,Kehrtwende in Drogenpolitik - Der lange Weg zu...,cannabis


## Save as Dataset

In [6]:
from datasets import Dataset, Features, ClassLabel, Value, Sequence, DatasetDict

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df['topics'] = df['topic'].apply(lambda topic: {topic: {'is_topic_bool': True}})
df.head()

,id,url,topic_folder,date_folder,filename,full_path,extracted_text,topic,topics
0,0,https://www.wa.de/nordrhein-westfalen/cannabis...,cannabis_html,Feb23,Legalisierung von Cannabis_ Hanf aus NRW könnt...,cannabis_html/Feb23/Legalisierung von Cannabis...,Legalisierung von Cannabis: Hanf aus NRW könnt...,cannabis,{'cannabis': {'is_topic_bool': True}}
1,1,https://www.ak-kurier.de/akkurier/www/artikel/...,cannabis_html,Feb23,Wann wird Cannabis in Deutschland legal_ _ AK-...,cannabis_html/Feb23/Wann wird Cannabis in Deut...,Wann wird Cannabis in Deutschland legal? | AK-...,cannabis,{'cannabis': {'is_topic_bool': True}}
2,2,https://www.aponet.de/artikel/weniger-alkoholp...,cannabis_html,Feb23,Weniger Alkoholprobleme bei Legalisierung von ...,cannabis_html/Feb23/Weniger Alkoholprobleme be...,Weniger Alkoholprobleme bei Legalisierung von ...,cannabis,{'cannabis': {'is_topic_bool': True}}
3,3,https://www.nordbayern.de/panorama/norwegische...,cannabis_html,Feb23,Norwegische Studie_ Legalisierung von Cannabis...,cannabis_html/Feb23/Norwegische Studie_ Legali...,Norwegische Studie: Legalisierung von Cannabis...,cannabis,{'cannabis': {'is_topic_bool': True}}
4,4,https://www.deutschlandfunk.de/legalisierung-c...,cannabis_html,Feb23,Kehrtwende in Drogenpolitik - Der lange Weg zu...,cannabis_html/Feb23/Kehrtwende in Drogenpoliti...,Kehrtwende in Drogenpolitik - Der lange Weg zu...,cannabis,{'cannabis': {'is_topic_bool': True}}


In [8]:
from urllib.parse import urlparse


def extract_domain(url: str) -> str:
    """
    Extracts the domain from a given URL. Prepends 'http://' if no scheme is found to ensure correct parsing. Returns an empty string if the URL is invalid or empty.
    """
    if not url:
        return ""
    
    if not urlparse(url).scheme:
        url = "http://" + url
    
    return urlparse(url).netloc.split('/')[0]

print(extract_domain("www.example.com/path/to/resource"))  # Output: www.example.com
print(extract_domain(""))

www.example.com



In [9]:
df['topics'] = df['topic'].apply(lambda topic: {topic: {'is_topic_bool': True}})
                                            
# Transform the DataFrame to match the desired Hugging Face Dataset format
transformed_data = {
    '_id': df['id'].apply(lambda x: 'dummy_id_' + str(x)),
    'batch_id': 99999,  
    'domain': [extract_domain(url) for url in df['url'].to_list()],  # Extract domain from URL
    'view_url': df['url'], 
    'lang': 'de', 
    'text': df['extracted_text'],
    'text_length': df['extracted_text'].apply(len),
    'word_count': df['extracted_text'].apply(lambda x: len(x.split())),
    'topics': df['topics'].tolist(),
    'topic': df['topic'],  
    'category': "buff",
    'good_for_training': 'True', 
    'good_for_augmentation': 'True', 
    'annotation_type': 'buff',
    'is_topic': True,
    'label': 1, 
}

# Create a new DataFrame that aligns with the Hugging Face Dataset format
hf_df = pd.DataFrame(transformed_data)

# Convert the pandas DataFrame to a Hugging Face dataset
hf_dataset = Dataset.from_pandas(hf_df)

print(hf_dataset)

Dataset({
    features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'topics', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label'],
    num_rows: 311
})


In [10]:
hf_dataset[0]

{'_id': 'dummy_id_0',
 'batch_id': 99999,
 'domain': 'www.wa.de',
 'view_url': 'https://www.wa.de/nordrhein-westfalen/cannabis-legalisierung-nrw-hanf-gras-ibbenbueren-entkriminalisierung-anbau-drogen-polizei-kritik-92064527.html',
 'lang': 'de',
 'text': 'Legalisierung von Cannabis: Hanf aus NRW könnte Problem lösen Hamm NRW Lokales Politik Sport Stellenmarkt ePaper Kategorien Hamm NRW Münster Essen Lokales Werne Bönen Bergkamen Drensteinfurt Welver Sport Hamm Bönen Drensteinfurt Fußball WA-Tipp Tabellen Bundesliga-Tippspiel Verbraucher Rückrufe & Warnungen Testberichte Supermärkte & Discounter Abo Print-Angebote Digital-Angebote Kombi-Angebote Abo-Services Wirtschaft Politik Karl Lauterbach Fotos & Videos Kultur Veranstaltungen Leben Auto Reise Karriere Gesundheit Wohnen Digital Tiere Geld Games Serien Einfach Tasty Buchtipps Anzeigen Gewerbliche Kunden Private Anzeigen Mediadaten Ansprechpartner Stellenanzeigen Autoanzeigen Traueranzeigen Über uns Ansprechpartner Impressum Jobs AGB D

In [11]:
hf_dataset.save_to_disk('../data/tmp/processed_dataset_all_topics_buff')

Saving the dataset (1/1 shards): 100%|██████████| 311/311 [00:00<00:00, 27737.86 examples/s]


## Save per Topic

In [12]:
topic = "cannabis"
filtered_dataset = hf_dataset.filter(lambda example: example['topic'] == topic)
constant_dict = {topic: {"is_topic_bool": True}}
filtered_dataset = filtered_dataset.map(lambda example: {"topics": constant_dict}, batched=False)
filtered_dataset = filtered_dataset.remove_columns(['label'])
filtered_dataset.save_to_disk(f'../data/tmp/processed_dataset_{topic}_buff')

Saving the dataset (1/1 shards): 100%|██████████| 150/150 [00:00<00:00, 20101.14 examples/s]


In [13]:
filtered_dataset[0]

{'_id': 'dummy_id_0',
 'batch_id': 99999,
 'domain': 'www.wa.de',
 'view_url': 'https://www.wa.de/nordrhein-westfalen/cannabis-legalisierung-nrw-hanf-gras-ibbenbueren-entkriminalisierung-anbau-drogen-polizei-kritik-92064527.html',
 'lang': 'de',
 'text': 'Legalisierung von Cannabis: Hanf aus NRW könnte Problem lösen Hamm NRW Lokales Politik Sport Stellenmarkt ePaper Kategorien Hamm NRW Münster Essen Lokales Werne Bönen Bergkamen Drensteinfurt Welver Sport Hamm Bönen Drensteinfurt Fußball WA-Tipp Tabellen Bundesliga-Tippspiel Verbraucher Rückrufe & Warnungen Testberichte Supermärkte & Discounter Abo Print-Angebote Digital-Angebote Kombi-Angebote Abo-Services Wirtschaft Politik Karl Lauterbach Fotos & Videos Kultur Veranstaltungen Leben Auto Reise Karriere Gesundheit Wohnen Digital Tiere Geld Games Serien Einfach Tasty Buchtipps Anzeigen Gewerbliche Kunden Private Anzeigen Mediadaten Ansprechpartner Stellenanzeigen Autoanzeigen Traueranzeigen Über uns Ansprechpartner Impressum Jobs AGB D

In [14]:
topic = "kinder"
filtered_dataset = hf_dataset.filter(lambda example: example['topic'] == topic)
constant_dict = {topic: {"is_topic_bool": True}}
filtered_dataset = filtered_dataset.map(lambda example: {"topics": constant_dict}, batched=False)
filtered_dataset = filtered_dataset.remove_columns(['label'])
filtered_dataset.save_to_disk(f'../data/tmp/processed_dataset_{topic}_buff')

Saving the dataset (1/1 shards): 100%|██████████| 76/76 [00:00<00:00, 10583.94 examples/s]


In [15]:
topic = "energie"
filtered_dataset = hf_dataset.filter(lambda example: example['topic'] == topic)
constant_dict = {topic: {"is_topic_bool": True}}
filtered_dataset = filtered_dataset.map(lambda example: {"topics": constant_dict}, batched=False)
filtered_dataset = filtered_dataset.remove_columns(['label'])
filtered_dataset.save_to_disk(f'../data/tmp/processed_dataset_{topic}_buff')

Saving the dataset (1/1 shards): 100%|██████████| 85/85 [00:00<00:00, 10530.98 examples/s]
